# Set environmental variables for local development

In [ ]:
import orjson, os
with open("local.settings.json") as f:
    os.environ.update(orjson.loads(f.read())["Values"])

# Get OnSpot Queue Stats
Legacy mode

In [ ]:
from libs.openapi.clients import OnSpotAPI
OSA = OnSpotAPI(production=True)
stat = OSA.createRequest(("/status/queue", "get"))
_, data, _ = await stat.request()
data

# Register SQL data provider
Note: this may take about 15-30 seconds to complete as it waits for the SQL database to be available and scans the provider metadata.

In [ ]:
from libs.data import from_bind, register_binding

register_binding(
    "keystone",
    "Structured",
    "sql",
    url=os.environ["DATABIND_SQL_KEYSTONE"],
    schemas=["public"],
)

# Instantiate data provider session

In [ ]:
from libs.data.structured.sqlalchemy import SQLAlchemyStructuredProvider

provider: SQLAlchemyStructuredProvider = from_bind("keystone")
tables = provider.models["public"]
session = provider.connect()

# Query provider session for data (location geoframes)

In [ ]:
from sqlalchemy.orm.query import Query

query: Query = session.query(
    tables["TargetingGeoFrame"].id,
    tables["TargetingGeoFrame"].polygon,
).filter(tables["TargetingGeoFrame"].polygon != None)
print(str(query.statement.compile(compile_kwargs={"literal_binds": True})))

# Format the OnSpot requests

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import uuid, os

# Relative date range
# now = datetime.utcnow()
# today = datetime(now.year, now.month, now.day)
# end = today - relativedelta(days=2)
# start = end - relativedelta(days=75)

# Static data range
start = datetime(2024,4,10)
end = datetime(2024,4,19)

requests = [
    {
        **row.polygon,
        "features": [
            {
                **feature,
                "properties": {
                    "name": row.id,
                    "start": start.isoformat(),
                    "end": end.isoformat(),
                    "hash": False,
                },
            }
            for feature in row.polygon.get("features", [])
        ]
    }
    for row in query
]

# Inject callback and outputLocation data

In [ ]:
from azure.storage.blob import (
    ContainerClient,
    ContainerSasPermissions,
    generate_container_sas,
)

ingress = {
    "instance_id": (instance_id := uuid.uuid4().hex),
    "conn_str": "ONSPOT_CONN_STR"
    if "ONSPOT_CONN_STR" in os.environ.keys()
    else "AzureWebJobsStorage",
    "container": os.environ.get("ONSPOT_CONTAINER", "dashboard"),
    "outputPath": f"oneoff/{instance_id}/devices",
    # "endpoint": "/save/geoframe/all/devices",
    "endpoint": "/geoframe/all/devices"
}

if ingress["endpoint"].startswith("/save/"):
    container = ContainerClient.from_connection_string(
        os.environ[ingress["conn_str"]]
        if ingress.get("conn_str", None) in os.environ.keys()
        else os.environ["AzureWebJobsStorage"],
        container_name=ingress.get("container", "general"),
    )
    if not container.exists():
        container.create_container()
    sas_token = generate_container_sas(
        account_name=container.credential.account_name,
        account_key=container.credential.account_key,
        container_name=container.container_name,
        permission=ContainerSasPermissions(write=True, read=True),
        expiry=datetime.utcnow() + relativedelta(days=2),
    )
event_url = "https://webhook.site/7faabe52-9b54-4c33-82c8-ada4b90f8530"

for request in requests:
    if request.get("type", None) == "FeatureCollection":
        for feature in request["features"]:
            feature["properties"]["callback"] = event_url.replace(
                "{eventName}", uuid.uuid4().hex
            )
            if ingress["endpoint"].startswith("/save/"):
                feature["properties"]["outputLocation"] = (
                    container.url.replace("https://", "az://")
                    + "/{}?".format(ingress.get("outputPath", ingress["instance_id"]))
                    + sas_token
                )

In [ ]:
from libs.openapi.clients.onspot import OnSpot
req = OnSpot[(ingress["endpoint"], "post")]
for request in requests:
    display(req(request))
    break # Comment this line to process the entire list of requests

# Address to Device ID

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import uuid, os

# Relative date range
now = datetime.utcnow()
today = datetime(now.year, now.month, now.day)
end = today - relativedelta(days=2)
start = end - relativedelta(days=75)

source_url = "https://esquireautoaudience.blob.core.windows.net/..."

requests = [
    {
        "hash": False,
        "name": uuid.uuid4().hex,
        "fileName": uuid.uuid4().hex + ".csv",
        "fileFormat": {
            "delimiter": ",",
            "quoteEncapsulate": True,
        },
        "mappings": {
            "street": ["address"],
            "city": ["city"],
            "state": ["state"],
            "zip": ["zipCode"],
            "zip4": ["plus4Code"],
        },
        "matchAcceptanceThreshold": 29.9,
        "sources": [source_url.replace("https://", "az://")],
    }
]


ingress = {
    "instance_id": (instance_id := uuid.uuid4().hex),
    "conn_str": "ONSPOT_CONN_STR"
    if "ONSPOT_CONN_STR" in os.environ.keys()
    else "AzureWebJobsStorage",
    "container": os.environ.get("ONSPOT_CONTAINER", "dashboard"),
    "outputPath": f"oneoff/{instance_id}/devices",
    "endpoint": "/save/addresses/all/devices",
}

from azure.storage.blob import (
    ContainerClient,
    ContainerSasPermissions,
    generate_container_sas,
)

if ingress["endpoint"].startswith("/save/"):
    container = ContainerClient.from_connection_string(
        os.environ[ingress["conn_str"]]
        if ingress.get("conn_str", None) in os.environ.keys()
        else os.environ["AzureWebJobsStorage"],
        container_name=ingress.get("container", "general"),
    )
    if not container.exists():
        container.create_container()
    sas_token = generate_container_sas(
        account_name=container.credential.account_name,
        account_key=container.credential.account_key,
        container_name=container.container_name,
        permission=ContainerSasPermissions(write=True, read=True),
        expiry=datetime.utcnow() + relativedelta(days=2),
    )
event_url = "https://webhook.site/60930832-3cb7-4a88-b845-3e76cc398fe8"

for request in requests:
    if ingress["endpoint"].startswith("/save/"):
        request["callback"] = event_url
        request["outputLocation"] = (
            container.url.replace("https://", "az://")
            + "/{}?".format(ingress.get("outputPath", ingress["instance_id"]))
            + sas_token
        )

from libs.openapi.clients.onspot import OnSpot
req = OnSpot[(ingress["endpoint"], "post")]
for request in requests:
    display(req(request))